In [ ]:
import pandas as pd


df_passes = pd.read_csv("df_passes.csv", index_col=0)
df_passes.head()

## Per player info

In [ ]:
df_passes.groupby("player_name").size().to_frame("num_passes")

In [ ]:
df_passes.groupby("player_name").agg({"vaep_value": "mean"})

## Per player pair info

In [ ]:
df_passes["pair_key"] = df_passes.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)

In [ ]:
df_passes.groupby("pair_key").size().to_frame("num_passes").head(20)

In [ ]:
df_passes.groupby("pair_key").agg({"vaep_value": "mean"}).head(20)